In [1]:
import random
import csv
import logging

from shop_crawler import *
from selenium_helper import *
import common_actors

from contextlib import contextmanager

### Prepare different urls to analyze

In [2]:
# All urls
all_urls = []
with open('../resources/pvio_vio_us_ca_uk_sample1.csv', 'r') as f:
    rows = csv.reader(f)
    for row in rows:
        url = row[0]
        if url:
            all_urls.append(url)

# Random sample urls
random.seed(1)
sample_urls = random.sample(all_urls, 100)

# Some good urls to analyze by hands
good_urls = [
    'curlebotanicals.com',
    'theglamourshop.com',
    'vape-fuel.com',
    'firstfitness.com',
    'sandlakedermatology.com',
    'getwaave.com',
    'dixieems.com',
    'jonessurgical.com',
    'srandd.com',
    'ambarygardens.com',
    'anabolicwarfare.com'
]


### Mock user and payment information

In [3]:
user_info = UserInfo(
    first_name = 'John',
    last_name = 'Smith',
    country = 'United States',
    home = 34,
    street = 'Ocean drive',
    city = 'Miami',
    zip = '33125',
    state = 'FLorida',
    
    phone = '1231232',
    email = 'john@service.com'
)

billing_info = PaymentInfo(
    card_number = '1413232312312321',
    expire_date_year = 2020,
    expire_date_month = 12,
    cvc = '123'
)

In [4]:
selenium_path = '/home/aleksei/dist/selenium/chromedriver'

@contextmanager
def get_crawler(headless=True):
    global user_info, billing_info, selinium_path
    crawler = ShopCrawler(user_info, billing_info, selenium_path, headless=headless)
    common_actors.add_crawler_extensions(crawler)
    
    yield crawler

def get_driver(headless=True):
    global selenium_path   
    return create_chrome_driver(selenium_path, headless=headless)


### Set up logging level

In [5]:
logger = logging.getLogger('shop_crawler')
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

### Start crawling for every url

In [10]:
results = []
with get_crawler(headless=False) as crawler:
    for url in sample_urls:
        print('\n\n{}'.format(url))
        status = crawler.crawl(url, 30)
        print(status)
        results.append(status)




vaporworld.biz
Status: "Time Out" after processing url "http://vaporworld.biz"


cvsciences.com


/home/aleksei/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2018-07-11 12:41:53,247 shop_crawler INFO     processing state: new
2018-07-11 12:41:53,248 shop_crawler INFO     handler <common_actors.AddToCart object at 0x7f4605c1d6d8>
2018-07-11 12:41:54,259 shop_crawler INFO     new_state new, url https://cvsciences.com/
2018-07-11 12:41:54,260 shop_crawler INFO     handler <common_actors.GoogleForProductPage object at 0x7f4605a67128>
2018-07-11 12:41:56,568 shop_crawler INFO     new_state new, url https://cvsciences.com/
2018-07-11 12:41:56,569 shop_crawler INFO     handler <common_actors.ToProductPageLink object at 0x7f4605a1f6a0>
2018-07-11 12:41:57,692 shop_crawler INFO     new_state new, url https://cvsciences.com/
2018-07-11 12:41:57,694 sh

Status: "Processing Finished at State" after processing url "cvsciences.com"


walkinmycloset.com
Status: "Not Available" after processing url "http://walkinmycloset.com"
 Domain walkinmycloset.com for sale


blackrosevapes.com
Status: "Not Available" after processing url "http://blackrosevapes.com"


fourriversbirdrescue.org
Status: "Not Available" after processing url "http://fourriversbirdrescue.org"


trelexa.org
Status: "Not Available" after processing url "http://trelexa.org"


webstersoccer.com


2018-07-11 12:42:09,973 shop_crawler INFO     processing state: new
2018-07-11 12:42:09,975 shop_crawler INFO     handler <common_actors.AddToCart object at 0x7f4605c1d6d8>
2018-07-11 12:42:15,752 shop_crawler INFO     new_state new, url http://www.lakefrontsc.com/
2018-07-11 12:42:15,754 shop_crawler INFO     handler <common_actors.GoogleForProductPage object at 0x7f4605a67128>
2018-07-11 12:42:17,478 shop_crawler INFO     new_state new, url http://www.lakefrontsc.com/
2018-07-11 12:42:17,479 shop_crawler INFO     handler <common_actors.ToProductPageLink object at 0x7f4605a1f6a0>
2018-07-11 12:42:21,084 shop_crawler DEBUG    Exception during checking element Traceback (most recent call last):
  File "/home/aleksei/work/projects/g2/trace_automation/crawling/selenium_helper.py", line 29, in can_click
    return element.is_enabled() and element.is_displayed()
  File "/home/aleksei/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webelement.py", line 159, in is_enabled
    

Status: "Processing Finished at State" after processing url "webstersoccer.com"


tampahumidor.com


...

2018-07-11 12:43:21,289 shop_crawler INFO     found 2 frames
2018-07-11 12:43:21,289 shop_crawler INFO     processing state: new
2018-07-11 12:43:21,290 shop_crawler INFO     handler <common_actors.AddToCart object at 0x7f4605c1d6d8>
2018-07-11 12:43:25,906 shop_crawler INFO     new_state new, url http://www.ladyfatemahtrust.org/
2018-07-11 12:43:25,907 shop_crawler INFO     handler <common_actors.GoogleForProductPage object at 0x7f4605a67128>
2018-07-11 12:43:27,872 shop_crawler INFO     new_state new, url http://www.ladyfatemahtrust.org/
2018-07-11 12:43:27,876 shop_crawler INFO     handler <common_actors.ToProductPageLink object at 0x7f4605a1f6a0>
2018-07-11 12:43:33,688 shop_crawler INFO     new_state new, url http://www.ladyfatemahtrust.org/
2018-07-11 12:43:33,690 shop_crawler INFO     handler <common_actors.ToShopLink object at 0x7f4605938358>
2018-07-11 12:43:37,870 shop_crawler DEBUG    clicking element: <a target="_blank" href="https://itunes.apple.com/us/app/lfct/id113233082

Status: "Processing Finished at State" after processing url "ladyfatemahtrust.org"


the420line.com


2018-07-11 12:43:49,378 shop_crawler ERROR    Unexpected exception during processing http://the420line.com
Traceback (most recent call last):
  File "/home/aleksei/work/projects/g2/trace_automation/crawling/shop_crawler.py", line 262, in crawl
    status = ShopCrawler.get(driver, url, wait_response_seconds)
  File "/home/aleksei/work/projects/g2/trace_automation/crawling/shop_crawler.py", line 197, in get
    driver.get(url)
  File "/home/aleksei/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 326, in get
    self.execute(Command.GET, {'url': url})
  File "/home/aleksei/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 312, in execute
    response = self.command_executor.execute(driver_command, params)
  File "/home/aleksei/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/remote_connection.py", line 472, in execute
    return self._request(command_info[0], url, body=data)
  File "/home/aleksei/anaconda

Status: "Processing Finished at State" after processing url "the420line.com"


allsquared.com


CannotSendRequest: Request-sent

In [ ]:
states = {}
for status in results:
    if isinstance(status, ProcessingStatus):
        states[status.state] = states.get(status.state, 0) + 1
        
print(states)

### Test Empty cart processing

In [ ]:
# Make it True if you want to test empty cart detection
test_empty_cart = False

checkout_pages = [
    'http://vape-fuel.com/checkout/cart/',
    'https://store-bb7f1.mybigcommerce.com/cart.php',
    'https://www.firstfitness.com/index.php?pg=signup',
    'https://store.sandlakedermatology.com/checkout/cart/',
    'https://www.dixieems.com/cart.asp',
    'http://www.jonessurgical.com/quick-order'    
]

if test_empty_cart:
    driver = get_driver()
    
    for url in checkout_pages:
        driver.get(url)
        time.sleep(30)
        
        is_empty_cart = common_actors.is_empty_cart(driver)
        print('url: {}, is empty cart: {}'.format(url, is_empty_cart))
        assert is_empty_cart

    driver.quit()

In [ ]:
test_broken_sites = False

not_available = [
    'juicyliquid.com',
    'gotbloody.com',
    'trelexa.org'
]

error = [
    'tampahumidor.com',
    'seriouscigars.com'    
]

timeout = [
    'moneynetint.com'
]

domains_for_sale = [
    'walkinmycloset.com'
]
        

if test_broken_sites:
    with get_crawler() as crawler:
        for url in domains_for_sale:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, NotAvailable), '{} must be not available'.format(url)
        
        for url in timeout:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, Timeout) or isinstance(status, NotAvailable), \
                '{} must be not timed out'.format(url)

        for url in not_available:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, NotAvailable), '{} must be not available'.format(url)

        for url in error:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, RequestError), '{} must produce an error'.format(url)
        
        